<a href="https://colab.research.google.com/github/AjayAndData/my_refernce_folder/blob/main/3cb/AMX_classfication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>